# problem3,a

In [19]:
import pandas as pd
import numpy as np
from numpy import *

import matplotlib.pyplot as plt

def featuresRead(filename):
    fr = open(filename)
    arrayLines = fr.readlines()         #get the number of lines in the file
    numberLines = len(arrayLines)
    returnMat = zeros((numberLines,3))        #prepare matrix to return
    index = 0
    for line in arrayLines:
        line = line.strip()
        listFromLine = line.split(' ')
        returnMat[index,:] = listFromLine[0:3]
        index += 1
    return returnMat

def labelsRead(filename):
    fr = open(filename)
    arrayLines = fr.readlines()         #get the number of lines in the file
    numberLines = len(arrayLines)
    returnMat = zeros((numberLines,1))        #prepare matrix to return
    index = 0
    for line in arrayLines:
        line = line.strip()
        listFromLine = line.split(' ')
        returnMat[index,:] = listFromLine[0:1]
        index += 1
    return returnMat

#load the train feature and train lables
Xtrain = featuresRead('train-features.txt')
ytrain = labelsRead('train-labels.txt')
Xtest = featuresRead('test-features.txt')
ytest = labelsRead('test-labels.txt')

X_message = int(max(Xtrain[:,0]))
X_word_id = int(max(Xtrain[:,1]))
X_line = Xtrain.shape[0]

Xtest_message = int(max(Xtest[:,0]))
Xtest_word_id = int(max(Xtest[:,1]))
Xtest_line = Xtest.shape[0]

i=0
wordgood = np.zeros( X_word_id )
wordbad = np.zeros( X_word_id )
for i in range(X_line): # every line of Xtrain
    j=int(Xtrain[i][0])-1
    k=int(Xtrain[i][1])-1
    if(ytrain[j][0]==0):
        wordgood[k] = wordgood[k] + int(Xtrain[i][2])
    else:
        wordbad[k] = wordbad[k] + int(Xtrain[i][2])

Pwordgood = np.zeros( X_word_id )
Pwordbad = np.zeros( X_word_id )
k=0

#sum_word=Xtrain.sum(axis=0)[2]
sum_wordgood =sum(wordgood)
sum_wordbad =sum(wordbad)
for k in range(X_word_id):
    Pwordgood[k]=log((wordgood[k]+0.01)/(sum_wordgood+X_word_id))
    Pwordbad[k]=log((wordbad[k]+0.01)/(sum_wordbad+X_word_id))

Pbad = log(sum(ytrain)/ len(ytrain)) 
Pgood =log(1- Pbad)   

from sklearn import metrics
import math
y_pred=[]
Py_pred=[]
i=0
j=1
y_predgood = Pgood
y_predbad = Pbad
for i in range(X_line):   #X_line

    if(Xtrain[i][0]==j):
        word=int(Xtrain[i][1])-1
        y_predgood = y_predgood + Pwordgood[word]*Xtrain[i][2]
        y_predbad = y_predbad + Pwordbad[word]*Xtrain[i][2]
    else:
        j=j+1

        if(y_predgood>y_predbad):
            y_pred.append([0])
        else:
            y_pred.append([1])
        
        Py_pred.append(y_predgood/(y_predgood+y_predbad))    
        y_predgood = Pgood
        y_predbad = Pbad
# add the last ypred    
if(y_predgood>y_predbad):
    y_pred.append([0])
else:
    y_pred.append([1])
Py_pred.append((y_predgood-y_predbad)/(y_predgood+y_predbad))

#compute the AUC
fpr, tpr, thresholds = metrics.roc_curve(ytrain[:,0], Py_pred)
AUCtrain = metrics.auc(fpr, tpr)
print('the AUC on the training set is:',AUCtrain)

#compute the classification error
i=0
class_error = 0
for i in range(X_message):
    if(ytrain[i][0]!=y_pred[i][0]):
        class_error=class_error+1
Pclass_error = class_error/X_message
print('the classification error rate of training is:',Pclass_error)
print('the wrong prediction number in training set is:',class_error)

the AUC on the training set is: 0.996767346939
the classification error rate of training is: 0.004285714285714286
the wrong prediction number in training set is: 3


# Problem 3 b

In [24]:
ytest_pred=[]
Pytest_pred=[]
i=0
j=1
ytest_predgood = Pgood
ytest_predbad = Pbad
for i in range(Xtest_line):
    
    if(Xtest[i][0]==j):
        word=int(Xtest[i][1])-1
        ytest_predgood = ytest_predgood+ Pwordgood[word]*Xtest[i][2]
        ytest_predbad = ytest_predbad+ Pwordbad[word]*Xtest[i][2]
    else:
        j=j+1
        
        if(ytest_predgood > ytest_predbad):
            ytest_pred.append([0])
        else:
            ytest_pred.append([1])
        Pytest_pred.append(ytest_predgood/(ytest_predgood+ytest_predbad))     
        ytest_predgood = Pgood
        ytest_predbad = Pbad

# add the last ypred    
if(ytest_predgood>ytest_predbad):
    ytest_pred.append([0])
else:
    ytest_pred.append([1])
Pytest_pred.append(ytest_predgood/(ytest_predgood+ytest_predbad)) 

fpr, tpr, thresholds = metrics.roc_curve(ytest[:,0], Pytest_pred)
AUCtrain_test = metrics.auc(fpr, tpr)
print('the AUC on the test set is:',AUCtrain_test)

i=0
class_error_test = 0
for i in range(Xtest_message):
    if(ytest[i][0]!=ytest_pred[i][0]):
        class_error_test=class_error_test+1
Pclass_error_test = class_error_test/Xtest_message
print('the classification error rate of test is:',Pclass_error_test)
print('the wrong prediction number in test set is:',class_error_test)

the AUC on the test set is: 0.998520710059
the classification error rate of test is: 0.015384615384615385
the wrong prediction number in test set is: 4
